In [1]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [2]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [5]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [6]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
#device = 'cpu'


device(type='cuda', index=0)

In [8]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.7308, 0.3672, 0.6496],
        [0.8686, 0.6107, 0.5251],
        [0.0569, 0.2399, 0.3190],
        [0.1135, 0.1350, 0.3536],
        [0.9651, 0.3276, 0.9218]], device='cuda:0')


In [9]:
x.get_device()

0

In [10]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [11]:
torch.cuda.memory_allocated()

0

In [12]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Sun Feb 19 03:52:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   34C    P0    36W / 250W |   1172MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [13]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
# Case2: codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000 for the model 'EleutherAI/gpt-neo-2.7B' <-- MEMORY CONSTRAINTS
# Case3: codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000 for the model 'EleutherAI/gpt-neo-1.3B'
# Case4: codesearch_tesbed_microsoft-CodeGPT-small-py_10000 for the model 'microsoft/CodeGPT-small-py'
# Case5: codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_10000 for the model 'microsoft/CodeGPT-small-py-adaptedGPT2'

def params(): 
    return {
            'big_table_path' : '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_10000.csv',
            'hf_model' : 'microsoft/CodeGPT-small-py-adaptedGPT2',
            #'hf_model' : 'microsoft/CodeGPT-small-py',
            'model_name': 'CodeGPT-small-py-adaptedGPT2_10000_',
            'callbacks' : '../data/callbacks-CodeGPT-small-py-adaptedGPT2_10000_',
            'batch': 1 
}

In [14]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_10000.csv'

# Data Upload

In [15]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [16]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,421.205700
std,369.798147
min,41.000000
25%,169.000000
50%,287.000000
75%,543.000000
max,2038.000000


In [17]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def plot_pattern(pattern, rule_number, filenam...","[('def', 'def', 'function_definition'), ('plot...","[(4299, 'def', 'function_definition'), (7110, ...","[4299, 7110, 62, 33279, 7, 33279, 11, 3896, 62...",160
1,"def parse_date(datestr: str, date_fmt: str) ->...","[('def', 'def', 'function_definition'), ('pars...","[(4299, 'def', 'function_definition'), (21136,...","[4299, 21136, 62, 4475, 7, 19608, 395, 81, 25,...",663
2,"def _render_rewrite(self, color=True):\n ...","[('def', 'def', 'function_definition'), ('_ren...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 13287, 62, 1809, 6525, 7, 944, 11...",211
3,"def clinvar_submission_header(submission_objs,...","[('def', 'def', 'function_definition'), ('clin...","[(4299, 'def', 'function_definition'), (5327, ...","[4299, 5327, 7785, 62, 7266, 3411, 62, 25677, ...",461
4,"def get_messages_by_line(messages):\n """"""Re...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 37348, 1095, 62, 1525, 62, 137...",76


In [18]:
data_pd_to_compare = pd.read_csv( 
                      '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv' , 
                      index_col=0
            )

In [19]:
data_pd_to_compare.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def child_object(self):\n """""" Get Task ...","[('def', 'def', 'function_definition'), ('chil...","[(4299, 'def', 'function_definition'), (1200, ...","[4299, 1200, 62, 15252, 7, 944, 2599, 198, 220...",92
1,"def manual_run(turn_into_run=True, store_meta_...","[('def', 'def', 'function_definition'), ('manu...","[(4299, 'def', 'function_definition'), (10107,...","[4299, 10107, 62, 5143, 7, 15344, 62, 20424, 6...",484
2,"def connectionLost(self, reason):\n """"""...","[('def', 'def', 'function_definition'), ('conn...","[(4299, 'def', 'function_definition'), (4637, ...","[4299, 4637, 31042, 7, 944, 11, 1738, 2599, 19...",94
3,"def as_thing_description(self):\n """"""\n...","[('def', 'def', 'function_definition'), ('as_t...","[(4299, 'def', 'function_definition'), (355, '...","[4299, 355, 62, 1197, 62, 11213, 7, 944, 2599,...",1080
4,"def sample_uniform_initial_state(parameter,\n ...","[('def', 'def', 'function_definition'), ('samp...","[(4299, 'def', 'function_definition'), (6291, ...","[4299, 6291, 62, 403, 6933, 62, 36733, 62, 521...",570


# Model Upload

In [20]:
#! pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [21]:
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git


In [22]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")


def testing_1():
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [23]:
#This code works for GPT-Neo
#from transformers import GPTNeoForCausalLM, GPT2Tokenizer
#model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
#tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [24]:
parameters['hf_model']

'microsoft/CodeGPT-small-py-adaptedGPT2'

In [25]:
#This code works for CodeGPT
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
#tokenizer.get_vocab()

In [27]:
#len( tokenizer.get_vocab() ) #todo an assert

In [28]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [29]:
len(prompts)

10000

In [30]:
#Making space for the tensors
#input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor [cast to list]
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [31]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [32]:
input_ids_list[0]

tensor([ 4299,  7110,    62, 33279,     7, 33279,    11,  3896,    62, 17618,
           11, 29472,  2599,   198,   220,   220,   220, 37227,  1345,  1747,
          281,  3557, 13951,  7559, 33279, 15506,   290,  7000,   262,  2939,
          739,   257,  1813,  7559, 34345, 15506,    13,   628,   220,   220,
          220,  1114, 34197, 14722,   262,  7559, 25135,    62, 17618, 15506,
          318,   635,  2672,    13,   628,   220,   220,   220, 37227,   198,
          220,   220,   220,   458,    83,    13, 26875,  3419,   198,   220,
          220,   220,   458,    83,    13,   320, 12860,     7, 33279,     8,
          198,   220,   220,   220,   458,    83,    13,    87, 18242, 10786,
        28780,  1400,  2637,     8,   198,   220,   220,   220,   458,    83,
           13,  2645,  9608, 10786,  7575,  5012, 11537,   198,   220,   220,
          220,   458,    83,    13,  7839, 10786,  8141,   351, 14330,  4064,
           82,     6,  4064,   965,     7, 25135,    62, 17618, 

In [33]:
#It should be same size
print(len(input_ids_list),len(prompts))
assert len(input_ids_list) == len(prompts)

10000 10000


In [34]:
data_pd.model_input_ids.values[0]

'[4299, 7110, 62, 33279, 7, 33279, 11, 3896, 62, 17618, 11, 29472, 2599, 198, 220, 220, 220, 37227, 1345, 1747, 281, 3557, 13951, 7559, 33279, 15506, 290, 7000, 262, 2939, 739, 257, 1813, 7559, 34345, 15506, 13, 628, 220, 220, 220, 1114, 34197, 14722, 262, 7559, 25135, 62, 17618, 15506, 318, 635, 2672, 13, 628, 220, 220, 220, 37227, 198, 220, 220, 220, 458, 83, 13, 26875, 3419, 198, 220, 220, 220, 458, 83, 13, 320, 12860, 7, 33279, 8, 198, 220, 220, 220, 458, 83, 13, 87, 18242, 10786, 28780, 1400, 2637, 8, 198, 220, 220, 220, 458, 83, 13, 2645, 9608, 10786, 7575, 5012, 11537, 198, 220, 220, 220, 458, 83, 13, 7839, 10786, 8141, 351, 14330, 4064, 82, 6, 4064, 965, 7, 25135, 62, 17618, 4008, 198, 220, 220, 220, 458, 83, 13, 21928, 5647, 7, 34345, 8, 198, 220, 220, 220, 1303, 489, 83, 13, 12860, 3419, 198, 220, 220, 220, 458, 83, 13, 19836, 3419]'

In [35]:
input_ids_list[0]

tensor([ 4299,  7110,    62, 33279,     7, 33279,    11,  3896,    62, 17618,
           11, 29472,  2599,   198,   220,   220,   220, 37227,  1345,  1747,
          281,  3557, 13951,  7559, 33279, 15506,   290,  7000,   262,  2939,
          739,   257,  1813,  7559, 34345, 15506,    13,   628,   220,   220,
          220,  1114, 34197, 14722,   262,  7559, 25135,    62, 17618, 15506,
          318,   635,  2672,    13,   628,   220,   220,   220, 37227,   198,
          220,   220,   220,   458,    83,    13, 26875,  3419,   198,   220,
          220,   220,   458,    83,    13,   320, 12860,     7, 33279,     8,
          198,   220,   220,   220,   458,    83,    13,    87, 18242, 10786,
        28780,  1400,  2637,     8,   198,   220,   220,   220,   458,    83,
           13,  2645,  9608, 10786,  7575,  5012, 11537,   198,   220,   220,
          220,   458,    83,    13,  7839, 10786,  8141,   351, 14330,  4064,
           82,     6,  4064,   965,     7, 25135,    62, 17618, 

In [36]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [37]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [38]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_10000.csv',
 'hf_model': 'microsoft/CodeGPT-small-py-adaptedGPT2',
 'model_name': 'CodeGPT-small-py-adaptedGPT2_10000_',
 'callbacks': '../data/callbacks-CodeGPT-small-py-adaptedGPT2_10000_',
 'batch': 1}

In [39]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        
        #output = [ model( 
        #    input_ids = i, 
        #    labels = i.type(torch.LongTensor).to(device) 
        #    ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
        output = []
        for i in input[n:n+batch]:
            print(i)
            output.append( 
                model( 
                    input_ids = i, 
                    labels = i.type(torch.LongTensor).to(device) 
                )
            )
        
        print(output)
        
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
            

    pass

In [40]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [41]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [42]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [43]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [44]:
#output_loss

In [45]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=29)

tensor([ 4299, 36545,  5460,  ...,   220,  1441, 10352], device='cuda:0',
       dtype=torch.int32)


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`